# The revised simplex method

## Table of contents

In [6]:
using LinearAlgebra
using SparseArrays
using SuiteSparse
using Random
using ArgParse

push!(LOAD_PATH, realpath("../julia_files"))

using lp_constants
using lp_utils
using lp_problem
using lp_read_mps

using BenchmarkTools
using Test

┌ Info: Precompiling lp_constants [top-level]
└ @ Base loading.jl:1423
┌ Info: Precompiling lp_utils [top-level]
└ @ Base loading.jl:1423
┌ Info: Precompiling lp_problem [top-level]
└ @ Base loading.jl:1423
┌ Info: Precompiling lp_read_mps [top-level]
└ @ Base loading.jl:1423


In [5]:
println(pwd())


/Users/roryyarr/Desktop/Linear Programming/lp_code/lp_julia/notebooks


In [ ]:
# Define the PreprocessedLPProblem struct
struct PreprocessedLPProblem
    original_problem::LPProblem
    reduced_problem::LPProblem
    removed_rows::Vector{Int}
    removed_cols::Vector{Int}
    row_ratios::Dict{Int, Tuple{Int, Float64}}
end

In [ ]:
function convert_to_standard_form(preprocessed_lp::PreprocessedLPProblem)
    lp = preprocessed_lp.reduced_problem
    c, A, b, l, u = lp.c, lp.A, lp.b, lp.l, lp.u
    m, n = size(A)
    
    # Handle lower and upper bounds
    for i in 1:n
        if l[i] > -Inf
            A = [A; zeros(1, n)]
            A[end, i] = -1
            push!(b, -l[i])
            push!(lp.constraint_types, 'L')
        end
        if u[i] < Inf
            A = [A; zeros(1, n)]
            A[end, i] = 1
            push!(b, u[i])
            push!(lp.constraint_types, 'L')
        end
    end
    
    m, n = size(A)
    new_A = spzeros(eltype(A), m, n + m)
    new_b = copy(b)
    new_c = [c; zeros(m)]
    
    for i in 1:m
        if lp.constraint_types[i] == 'L'
            new_A[i, :] = [A[i, :]; zeros(i-1); 1; zeros(m-i)]
        elseif lp.constraint_types[i] == 'G'
            new_A[i, :] = [-A[i, :]; zeros(i-1); 1; zeros(m-i)]
            new_b[i] = -new_b[i]
        elseif lp.constraint_types[i] == 'E'
            new_A[i, :] = [A[i, :]; zeros(m)]
        end
    end
    
    if !lp.is_minimize
        new_c = -new_c
    end

    # Update the reduced LPProblem within PreprocessedLPProblem
    reduced_lp = LPProblem(lp.is_minimize, new_c, new_A, new_b, lp.l, lp.u, lp.vars, lp.constraint_types)

    return PreprocessedLPProblem(preprocessed_lp.original_problem, reduced_lp, preprocessed_lp.removed_rows, preprocessed_lp.removed_cols, preprocessed_lp.row_ratios)
end
